## Real-Time Sentiment Analysis Using Spark Streaming

#### Set up streaming source

In [3]:
import sys
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.streaming.kinesis import KinesisUtils, InitialPositionInStream
from pyspark.sql.types import *

spark = SparkSession.builder\
                    .master("local")\
                    .appName("Structured Streaming - Twitter Sentiment")\
                    .getOrCreate()

pythonSchema = StructType() \
          .add("id", StringType(), True) \
          .add("tweet", StringType(), True) \
          .add("ts", StringType(), True)

awsAccessKeyId = "" # update the access key
awsSecretKey = ""   # update the secret key
kinesisStreamName = "twitter-data-stream"  # update the kinesis stream name (need to set up the stream first and ingest data)
kinesisRegion = "us-east-1"


kinesisDF = spark \
  .readStream \
  .format("kinesis") \
  .option("streamName", kinesisStreamName)\
  .option("region", kinesisRegion) \
  .option("initialPosition", "latest") \
  .option("format", "json") \
  .option("awsAccessKey", awsAccessKeyId)\
  .option("awsSecretKey", awsSecretKey) \
  .option("inferSchema", "true") \
  .load()


#### <img src='https://s3.amazonaws.com/weclouddata/images/logos/asterisk_1.png' width='2%'> Create Streaming DF

In [5]:
df = kinesisDF \
  .writeStream \
  .format("memory") \
  .outputMode("append") \
  .queryName("tweets")  \
  .start()

In [6]:
df.status

Out[7]: {'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

#### Explore the stream data in SQL

In [8]:
%sql

select partitionKey, cast(data as string) from tweets;

partitionKey,data
wizpampy,"[{""id"": ""1243933850968809472"", ""tweet"": ""b'RT @T9uRhHfR8jsSP57: \\xe0\\xb9\\x83\\xe0\\xb8\\x99\\xe0\\xb8\\x95\\xe0\\xb8\\xad\\xe0\\xb8\\x99\\xe0\\xb8\\x99\\xe0\\xb8\\xb5\\xe0\\xb9\\x89\\xe0\\xb8\\x9a\\xe0\\xb8\\xb2\\xe0\\xb8\\x87\\xe0\\xb8\\x88\\xe0\\xb8\\xb1\\xe0\\xb8\\x87\\xe0\\xb8\\xab\\xe0\\xb8\\xa7\\xe0\\xb8\\xb1\\xe0\\xb8\\x94\\xe0\\xb8\\x95\\xe0\\xb9\\x89\\xe0\\xb8\\xad\\xe0\\xb8\\x87\\xe0\\xb8\\x9b\\xe0\\xb8\\xaa\\xe0\\xb8\\x94\\xe0\\xb8\\x81\\xe0\\xb8\\xb1\\xe0\\xb8\\x9a\\xe0\\xb9\\x84\\xe0\\xb8\\xa7\\xe0\\xb8\\xa3\\xe0\\xb8\\xb1\\xe0\\xb8\\xaa \\xe0\\xb9\\x81\\xe0\\xb8\\x95\\xe0\\xb9\\x88\\xe0\\xb9\\x80\\xe0\\xb8\\x8a\\xe0\\xb8\\xb5\\xe0\\xb8\\xa2\\xe0\\xb8\\x87\\xe0\\xb9\\x83\\xe0\\xb8\\xab\\xe0\\xb8\\xa1\\xe0\\xb9\\x88\\xe0\\xb8\\x95\\xe0\\xb8\\xad\\xe0\\xb8\\x99\\xe0\\xb8\\x99\\xe0\\xb8\\xb5\\xe0\\xb9\\x89\\xe0\\xb8\\x84\\xe0\\xb8\\xb7\\xe0\\xb8\\xad \\xe0\\xb9\\x84\\xe0\\xb8\\x9f\\xe0\\xb8\\x9b\\xe0\\xb9\\x88\\xe0\\xb8\\xb2 \\xe0\\xb9\\x84\\xe0\\xb8\\xa7\\xe0\\xb8\\xa3\\xe0\\xb8\\xb1\\xe0\\xb8\\xaa\\xe0\\xb8\\x81\\xe0\\xb9\\x87\\xe0\\xb8\\x81\\xe0\\xb8\\xa5\\xe0\\xb8\\xb1\\xe0\\xb8\\xa7\\xe0\\xb9\\x81\\xe0\\xb8\\x96\\xe0\\xb8\\xa1\\xe0\\xb8\\x95\\xe0\\xb8\\xad\\xe0\\xb8\\x99\\xe0\\xb8\\x99\\xe0\\xb8\\xb5\\xe0\\xb9\\x89\\xe0\\xb8\\x94\\xe0\\xb9\\x89\\xe0\\xb8\\xa7\\xe0\\xb8\\xa2\\xe0\\xb8\\x84\\xe0\\xb9\\x88\\xe0\\xb8\\xb2\\xe0\\xb8\\x9d\\xe0\\xb8\\xb8\\xe0\\xb9\\x88\\xe0\\xb8\\x99\\xe0\\xb8\\x97\\xe0\\xb8\\xb5\\xe0\\xb9\\x88\\xe0\\xb8\\x89\\xe0\\xb8\\xb4\\xe0\\xb8\\x9a\\xe0\\xb8\\xab\\xe0\\xb8\\xb2\\xe0\\xb8\\xa2\\xe0\\xb8\\x82\\xe0\\xb8\\xb6\\xe0\\xb9\\x89\\xe0\\xb8\\x99\\xe0\\xb9\\x84\\xe0\\xb8\\x9b\\xe0\\xb8\\xad\\xe0\\xb8\\xb5\\xe0\\xb8\\x81 \\xe0\\xb9\\x80\\xe0\\xb8\\xad\\xe0\\xb8\\xb2\\xe0\\xb9\\x83\\xe0\\xb8\\x88\\xe2\\x80\\xa6'"", ""ts"": ""Sat Mar 28 16:11:55 +0000 2020""}]"
PragNews,"[{""id"": ""1243933850666823680"", ""tweet"": ""b'States affected by Coronavirus in India\\n\\n#IndiaFightsCorona \\n#Covid_19 #coronavirus https://t.co/V6Sp85u3gX'"", ""ts"": ""Sat Mar 28 16:11:55 +0000 2020""}]"
Amina_I_sheikh,"[{""id"": ""1243933849853149186"", ""tweet"": ""b\""@ABSCBNNews Watch #MuhammadShaikh, who has delivered more than 50 lectures on different topics of Qur'an in English\\xe2\\x80\\xa6 https://t.co/7GPuUE9Bwx\"""", ""ts"": ""Sat Mar 28 16:11:55 +0000 2020""}]"
mcezeh1,"[{""id"": ""1243933852273455104"", ""tweet"": ""b'RT @BayoOmoboriowo: Thank you for checking up brother. Here are some images from today, Pls stay safe also. https://t.co/yBHZr1w1Ux https:/\\xe2\\x80\\xa6'"", ""ts"": ""Sat Mar 28 16:11:56 +0000 2020""}]"
Afridi_huOO7,"[{""id"": ""1243933852567044096"", ""tweet"": ""b'RT @Humafayaz1: Can\\'t Wait For The Day To Hear\\n \""Corona Virus Is Finally Gone\""\\n#Covid_19 #SaturdayMotivation'"", ""ts"": ""Sat Mar 28 16:11:56 +0000 2020""}]"
beautiquebkk,"[{""id"": ""1243933852994654208"", ""tweet"": ""b'\\xe0\\xb9\\x82\\xe0\\xb8\\x9b\\xe0\\xb8\\xa3\\xe0\\xb8\\x94\\xe0\\xb8\\xb5 \\xe0\\xb9\\x84\\xe0\\xb8\\xa1\\xe0\\xb9\\x88\\xe0\\xb8\\x95\\xe0\\xb9\\x89\\xe0\\xb8\\xad\\xe0\\xb8\\x87\\xe0\\xb8\\xa3\\xe0\\xb8\\xad OTP\\n\\n\\xe0\\xb9\\x82\\xe0\\xb8\\x9b\\xe0\\xb8\\xa3\\xe0\\xb8\\xa5\\xe0\\xb8\\x94\\xe0\\xb8\\x97\\xe0\\xb9\\x89\\xe0\\xb8\\xb2\\xe0\\xb9\\x80\\xe0\\xb8\\x84\\xe0\\xb8\\xad\\xe0\\xb8\\xa3\\xe0\\xb9\\x8c\\xe0\\xb8\\x9f\\xe0\\xb8\\xb4\\xe0\\xb8\\xa7\\n#\\xe0\\xb8\\xa3\\xe0\\xb8\\xb2\\xe0\\xb8\\x84\\xe0\\xb8\\xb2\\xe0\\xb8\\x8a\\xe0\\xb8\\xb4\\xe0\\xb8\\xa5 \\xe0\\xb8\\x97\\xe0\\xb8\\xb8\\xe0\\xb8\\x81\\xe0\\xb8\\x9c\\xe0\\xb8\\xa5\\xe0\\xb8\\xb4\\xe0\\xb8\\x95\\xe0\\xb8\\xa0\\xe0\\xb8\\xb1\\xe0\\xb8\\x93\\xe0\\xb8\\x91\\xe0\\xb9\\x8c\\n#\\xe0\\xb8\\x96\\xe0\\xb8\\xb9\\xe0\\xb8\\x81\\xe0\\xb8\\x97\\xe0\\xb8\\xb5\\xe0\\xb9\\x88\\xe0\\xb8\\xaa\\xe0\\xb8\\xb8\\xe0\\xb8\\x94 \\xe0\\xb8\\x97\\xe0\\xb8\\xb5\\xe0\\xb9\\x88\\xe0\\xb9\\x80\\xe0\\xb8\\x84\\xe0\\xb8\\xa2\\xe0\\xb8\\xa1\\xe0\\xb8\\xb5\\xe0\\xb8\\xa1\\xe0\\xb8\\xb2\\n\\xe0\\xb9\\x82\\xe0\\xb8\\xad\\

In [9]:
tweets = spark.sql("select cast(data as string) from tweets")

In [10]:
tweets.printSchema()

root
-- data: string (nullable = true)

In [11]:
tweets.show(5, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
data |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[{"id": "1243933850968809472", "tweet": "b'RT @T9uRhHfR8jsSP57: \\xe0\\xb9\\x83\\xe0\\xb8\\x99\\xe0\\xb8\\x95\\xe0\\xb8\\xad\\xe0\\xb8\\x99\\xe0\\xb8\\x99\\xe0\\xb8\\xb5\\xe0\\xb9\\x89\\xe0\\xb8\\x9a\\xe0\\xb8\\xb2\\xe0\\xb8\\x87\\xe0\\xb8\\x88\\xe0\\xb8\\xb1\\xe0\\xb8\\x87\\xe0\\xb8\\xab\\xe0\\xb8\\xa7\\xe0\\xb8\\xb1\\xe0\\xb8\\x94\\xe0\\xb

In [12]:
tweets.show(5, truncate=True)

+--------------------+
 data|
+--------------------+
[{"id": "12378657...|
[{"id": "12378657...|
[{"id": "12378657...|
[{"id": "12378657...|
[{"id": "12378657...|
+--------------------+
only showing top 5 rows

In [13]:
tweets.count()

Out[65]: 29045

#### Parsing the Streaming Tweets

In [15]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *

import json

def parse_tweet(text):
    data = json.loads(text)
    id = data[0]['id']
    ts = data[0]['ts']
    tweet = data[0]['tweet'] 
    return (id, ts, tweet)
    
# Define your function
getID = UserDefinedFunction(lambda x: parse_tweet(x)[0], StringType())
getTs = UserDefinedFunction(lambda x: parse_tweet(x)[1], StringType())
getTweet = UserDefinedFunction(lambda x: parse_tweet(x)[2], StringType())

# Apply the UDF using withColumn
tweets = (tweets.withColumn('id', getID(col("data")))
               .withColumn('ts', getTs(col("data")))
               .withColumn('tweet', getTweet(col("data")))
         )


In [16]:
tweets.show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
data |id |ts |tweet |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [17]:
tweets.show(truncate=True)

+--------------------+-------------------+--------------------+--------------------+
 data| id| ts| tweet|
+--------------------+-------------------+--------------------+--------------------+
[{"id": "12439338...|1243933850968809472|Sat Mar 28 16:11:...|b'RT @T9uRhHfR8js...|
[{"id": "12439338...|1243933850666823680|Sat Mar 28 16:11:...|b'States affected...|
[{"id": "12439338...|1243933849853149186|Sat Mar 28 16:11:...|b"@ABSCBNNews Wat...|
[{"id": "12439338...|1243933852273455104|Sat Mar 28 16:11:...|b'RT @BayoOmobori...|
[{"id": "12439338...|1243933852567044096|Sat Mar 28 16:11:...|b'RT @Humafayaz1:...|
[{"id": "12439338...|1243933852994654208|Sat Mar 28 16:11:...|b'\xe0\xb9\x82\xe...|
[{"id": "12439338...|1243933852864823296|Sat Mar 28 16:11:...|b'Ben de 81 milyo...|
[{"id": "12439338...|1243933853300822016|Sat Mar 28 16:11:...|b'Citizens can al...|
[{"id": "12439338...|1243933853636534272|Sat Mar 28 16:11:...|b'RT @GamingGifte...|
[{"id": "12439338...|1243933853653151746|Sat Mar 28 16:11:...|b'RT @Stone_nayou...|
[{"id": "12439338...|1243933853657509891|Sat Mar 28 16:11:...|b'RT @VassyKapelo...|
[{"id": "12439338...|1243933853825273857|Sat Mar 28 16:11:...|b'RT @aotxotorena...|
[{"id": "12439338...|1243933854152429569|Sat Mar 28 16:11:...|b'@herstalle @cat...|
[{"id": "12439338...|1243933853921628160|Sat Mar 28 16:11:...|b'#NDTVirus Jo ki...|
[{"id": "12439338...|1243933854324396034|Sat Mar 28 16:11:...|b'RT @ufpol: Muy ...|
[{"id": "12439338...|1243933854752219138|Sat Mar 28 16:11:...|b'#IzquierdaObstr...|
[{"id": "12439338...|1243933855154950144|Sat Mar 28 16:11:...|b'RT @AABenedetti...|
[{"id": "12439338...|1243933855503056898|Sat Mar 28 16:11:...|b'RT @JefeVergara...|
[{"id": "12439338...|1243933855918305280|Sat Mar 28 16:11:...|b'RT @lavecinarub...|
[{"id": "12439338...|1243933856572596224|Sat Mar 28 16:11:...| b'@Nanchinho'|
+--------------------+-------------------+--------------------+--------------------+
only showing top 20 rows

In [18]:
tweets.printSchema()

root
-- data: string (nullable = true)
-- id: string (nullable = true)
-- ts: string (nullable = true)
-- tweet: string (nullable = true)

#### Sentiment Scoring on Streaming Tweets using TextBlob

In [20]:
import textblob

In [21]:
def get_sentiment(text):
    from textblob import TextBlob
    tweet = TextBlob(text)
    if tweet.sentiment.polarity < 0:
      sentiment = "negative"
    elif tweet.sentiment.polarity == 0:
        sentiment = "neutral"
    else:
        sentiment = "positive"
    return sentiment
  
# Define your function
getSentiment = UserDefinedFunction(lambda x: get_sentiment(x), StringType())

# Apply the UDF using withColumn
tweets = tweets.withColumn('sentiment', getSentiment(col("tweet")))

In [22]:
tweets.show()

+--------------------+-------------------+--------------------+--------------------+---------+
 data| id| ts| tweet|sentiment|
+--------------------+-------------------+--------------------+--------------------+---------+
[{"id": "12439338...|1243933850968809472|Sat Mar 28 16:11:...|b'RT @T9uRhHfR8js...| neutral|
[{"id": "12439338...|1243933850666823680|Sat Mar 28 16:11:...|b'States affected...| neutral|
[{"id": "12439338...|1243933849853149186|Sat Mar 28 16:11:...|b"@ABSCBNNews Wat...| positive|
[{"id": "12439338...|1243933852273455104|Sat Mar 28 16:11:...|b'RT @BayoOmobori...| positive|
[{"id": "12439338...|1243933852567044096|Sat Mar 28 16:11:...|b'RT @Humafayaz1:...| neutral|
[{"id": "12439338...|1243933852994654208|Sat Mar 28 16:11:...|b'\xe0\xb9\x82\xe...| neutral|
[{"id": "12439338...|1243933852864823296|Sat Mar 28 16:11:...|b'Ben de 81 milyo...| neutral|
[{"id": "12439338...|1243933853300822016|Sat Mar 28 16:11:...|b'Citizens can al...| neutral|
[{"id": "12439338...|1243933853636534272|Sat Mar 28 16:11:...|b'RT @GamingGifte...| neutral|
[{"id": "12439338...|1243933853653151746|Sat Mar 28 16:11:...|b'RT @Stone_nayou...| neutral|
[{"id": "12439338...|1243933853657509891|Sat Mar 28 16:11:...|b'RT @VassyKapelo...| neutral|
[{"id": "12439338...|1243933853825273857|Sat Mar 28 16:11:...|b'RT @aotxotorena...| neutral|
[{"id": "12439338...|1243933854152429569|Sat Mar 28 16:11:...|b'@herstalle @cat...| neutral|
[{"id": "12439338...|1243933853921628160|Sat Mar 28 16:11:...|b'#NDTVirus Jo ki...| neutral|
[{"id": "12439338...|1243933854324396034|Sat Mar 28 16:11:...|b'RT @ufpol: Muy ...| neutral|
[{"id": "12439338...|1243933854752219138|Sat Mar 28 16:11:...|b'#IzquierdaObstr...| neutral|
[{"id": "12439338...|1243933855154950144|Sat Mar 28 16:11:...|b'RT @AABenedetti...| neutral|
[{"id": "12439338...|1243933855503056898|Sat Mar 28 16:11:...|b'RT @JefeVergara...| neutral|
[{"id": "12439338...|1243933855918305280|Sat Mar 28 16:11:...|b'RT @lavecinarub...| neutral|
[{"id": "12439338...|1243933856572596224|Sat Mar 28 16:11:...| b'@Nanchinho'| neutral|
+--------------------+-------------------+--------------------+--------------------+---------+
only showing top 20 rows

In [23]:
tweets.printSchema()

root
-- data: string (nullable = true)
-- id: string (nullable = true)
-- ts: string (nullable = true)
-- tweet: string (nullable = true)
-- sentiment: string (nullable = true)

In [24]:
tweets.createOrReplaceTempView("tweets_parsed")
#tweets.groupBy('sentiment').count()

In [25]:
%sql
select sentiment, count(*) as covid19_canada_tweet from tweets_parsed group by sentiment

sentiment,covid19_canada_tweet
positive,7154
neutral,32258
negative,3356


In [26]:
import plotly.express as px

import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')

fig = px.line(df, x='Date', y='AAPL.High')
fig.show()

## Convert Spark Dataframe to Pandas Dataframe

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime

In [29]:
type(tweets)

Out[20]: pyspark.sql.dataframe.DataFrame

In [30]:
tweets.dtypes

Out[21]: [('data', 'string'),
 ('id', 'string'),
 ('ts', 'string'),
 ('tweet', 'string'),
 ('sentiment', 'string')]

In [31]:
tweets_pdf = tweets.toPandas()

In [32]:
tweets_pdf.head()

,data,id,ts,tweet,sentiment
0,"[{""id"": ""1243933850968809472"", ""tweet"": ""b'RT ...",1243933850968809472,Sat Mar 28 16:11:55 +0000 2020,b'RT @T9uRhHfR8jsSP57: \xe0\xb9\x83\xe0\xb8\x9...,neutral
1,"[{""id"": ""1243933850666823680"", ""tweet"": ""b'Sta...",1243933850666823680,Sat Mar 28 16:11:55 +0000 2020,b'States affected by Coronavirus in India\n\n#...,neutral
2,"[{""id"": ""1243933849853149186"", ""tweet"": ""b\""@A...",1243933849853149186,Sat Mar 28 16:11:55 +0000 2020,"b""@ABSCBNNews Watch #MuhammadShaikh, who has d...",positive
3,"[{""id"": ""1243933852273455104"", ""tweet"": ""b'RT ...",1243933852273455104,Sat Mar 28 16:11:56 +0000 2020,b'RT @BayoOmoboriowo: Thank you for checking u...,positive
4,"[{""id"": ""1243933852567044096"", ""tweet"": ""b'RT ...",1243933852567044096,Sat Mar 28 16:11:56 +0000 2020,b'RT @Humafayaz1: Can\'t Wait For The Day To H...,neutral


In [33]:
pd.to_datetime(tweets_pdf['ts'])

Out[26]: 0 2020-03-28 16:11:55+00:00
1 2020-03-28 16:11:55+00:00
2 2020-03-28 16:11:55+00:00
3 2020-03-28 16:11:56+00:00
4 2020-03-28 16:11:56+00:00
5 2020-03-28 16:11:56+00:00
6 2020-03-28 16:11:56+00:00
7 2020-03-28 16:11:56+00:00
8 2020-03-28 16:11:56+00:00
9 2020-03-28 16:11:56+00:00
10 2020-03-28 16:11:56+00:00
11 2020-03-28 16:11:56+00:00
12 2020-03-28 16:11:56+00:00
13 2020-03-28 16:11:56+00:00
14 2020-03-28 16:11:56+00:00
15 2020-03-28 16:11:56+00:00
16 2020-03-28 16:11:56+00:00
17 2020-03-28 16:11:56+00:00
18 2020-03-28 16:11:57+00:00
19 2020-03-28 16:11:57+00:00
20 2020-03-28 16:11:57+00:00
21 2020-03-28 16:11:57+00:00
22 2020-03-28 16:11:57+00:00
23 2020-03-28 16:11:57+00:00
24 2020-03-28 16:11:57+00:00
25 2020-03-28 16:11:57+00:00
26 2020-03-28 16:11:57+00:00
27 2020-03-28 16:11:57+00:00
28 2020-03-28 16:11:57+00:00
29 2020-03-28 16:11:57+00:00
 ... 
48962 2020-03-28 17:02:03+00:00
48963 2020-03-28 17:02:03+00:00
48964 2020-03-28 17:02:03+00:00
48965 2020-03-28 17:02:03+00:00
48966 2020-03-28 17:02:03+00:00
48967 2020-03-28 17:02:03+00:00
48968 2020-03-28 17:02:03+00:00
48969 2020-03-28 17:02:04+00:00
48970 2020-03-28 17:02:04+00:00
48971 2020-03-28 17:02:04+00:00
48972 2020-03-28 17:02:04+00:00
48973 2020-03-28 17:02:04+00:00
48974 2020-03-28 17:02:04+00:00
48975 2020-03-28 17:02:04+00:00
48976 2020-03-28 17:02:04+00:00
48977 2020-03-28 17:02:04+00:00
48978 2020-03-28 17:02:04+00:00
48979 2020-03-28 17:02:04+00:00
48980 2020-03-28 17:02:04+00:00
48981 2020-03-28 17:02:04+00:00
48982 2020-03-28 17:02:04+00:00
48983 2020-03-28 17:02:04+00:00
48984 2020-03-28 17:02:04+00:00
48985 2020-03-28 17:02:04+00:00
48986 2020-03-28 17:02:05+00:00
48987 2020-03-28 17:02:05+00:00
48988 2020-03-28 17:02:05+00:00
48989 2020-03-28 17:02:05+00:00
48990 2020-03-28 17:02:05+00:00
48991 2020-03-28 17:02:05+00:00
Name: ts, Length: 48992, dtype: datetime64[ns, UTC]

In [34]:
idx = pd.DatetimeIndex(pd.to_datetime(tweets_pdf['ts']))

In [35]:
idx

Out[28]: DatetimeIndex(['2020-03-28 16:11:55+00:00', '2020-03-28 16:11:55+00:00',
 '2020-03-28 16:11:55+00:00', '2020-03-28 16:11:56+00:00',
 '2020-03-28 16:11:56+00:00', '2020-03-28 16:11:56+00:00',
 '2020-03-28 16:11:56+00:00', '2020-03-28 16:11:56+00:00',
 '2020-03-28 16:11:56+00:00', '2020-03-28 16:11:56+00:00',
 ...
 '2020-03-28 17:02:04+00:00', '2020-03-28 17:02:04+00:00',
 '2020-03-28 17:02:04+00:00', '2020-03-28 17:02:04+00:00',
 '2020-03-28 17:02:05+00:00', '2020-03-28 17:02:05+00:00',
 '2020-03-28 17:02:05+00:00', '2020-03-28 17:02:05+00:00',
 '2020-03-28 17:02:05+00:00', '2020-03-28 17:02:05+00:00'],
 dtype='datetime64[ns, UTC]', name='ts', length=48992, freq=None)

In [36]:
type(idx)

Out[54]: pandas.core.indexes.datetimes.DatetimeIndex

In [37]:
len(pd.to_datetime(tweets_pdf['ts']))

Out[29]: 48992

In [38]:
ones = np.ones(len(pd.to_datetime(tweets_pdf['ts'])))

In [39]:
idx.shape[0]

Out[31]: 48992

In [40]:
# the actual series (at series of 1s for the moment) 
my_series = pd.Series(ones, index=idx)

In [41]:
# Resampling / bucketing into 1-minute buckets
per_minute = my_series.resample('1Min').sum().fillna(0) 

In [42]:
per_minute

Out[34]: ts
2020-03-28 16:11:00+00:00 58.0
2020-03-28 16:12:00+00:00 962.0
2020-03-28 16:13:00+00:00 973.0
2020-03-28 16:14:00+00:00 955.0
2020-03-28 16:15:00+00:00 1083.0
2020-03-28 16:16:00+00:00 1000.0
2020-03-28 16:17:00+00:00 974.0
2020-03-28 16:18:00+00:00 909.0
2020-03-28 16:19:00+00:00 927.0
2020-03-28 16:20:00+00:00 1018.0
2020-03-28 16:21:00+00:00 1036.0
2020-03-28 16:22:00+00:00 969.0
2020-03-28 16:23:00+00:00 1012.0
2020-03-28 16:24:00+00:00 984.0
2020-03-28 16:25:00+00:00 986.0
2020-03-28 16:26:00+00:00 945.0
2020-03-28 16:27:00+00:00 959.0
2020-03-28 16:28:00+00:00 931.0
2020-03-28 16:29:00+00:00 1031.0
2020-03-28 16:30:00+00:00 1089.0
2020-03-28 16:31:00+00:00 948.0
2020-03-28 16:32:00+00:00 1004.0
2020-03-28 16:33:00+00:00 963.0
2020-03-28 16:34:00+00:00 919.0
2020-03-28 16:35:00+00:00 982.0
2020-03-28 16:36:00+00:00 995.0
2020-03-28 16:37:00+00:00 1011.0
2020-03-28 16:38:00+00:00 956.0
2020-03-28 16:39:00+00:00 949.0
2020-03-28 16:40:00+00:00 1004.0
2020-03-28 16:41:00+00:00 957.0
2020-03-28 16:42:00+00:00 914.0
2020-03-28 16:43:00+00:00 909.0
2020-03-28 16:44:00+00:00 980.0
2020-03-28 16:45:00+00:00 989.0
2020-03-28 16:46:00+00:00 971.0
2020-03-28 16:47:00+00:00 957.0
2020-03-28 16:48:00+00:00 943.0
2020-03-28 16:49:00+00:00 1013.0
2020-03-28 16:50:00+00:00 966.0
2020-03-28 16:51:00+00:00 1050.0
2020-03-28 16:52:00+00:00 1040.0
2020-03-28 16:53:00+00:00 945.0
2020-03-28 16:54:00+00:00 917.0
2020-03-28 16:55:00+00:00 974.0
2020-03-28 16:56:00+00:00 954.0
2020-03-28 16:57:00+00:00 986.0
2020-03-28 16:58:00+00:00 948.0
2020-03-28 16:59:00+00:00 975.0
2020-03-28 17:00:00+00:00 1050.0
2020-03-28 17:01:00+00:00 940.0
2020-03-28 17:02:00+00:00 82.0
Freq: T, dtype: float64

In [43]:
# Plotting the series
%matplotlib inline

fig, ax = plt.subplots()
ax.grid(True)

ax.set_title("Tweet numbers")
interval = mdates.MinuteLocator(interval=10)
date_formatter = mdates.DateFormatter('%H:%M')

datemin = datetime(2020, 3, 28, 16, 00) 
datemax = datetime(2020, 3, 28, 17, 10)

ax.xaxis.set_major_locator(interval) 
ax.xaxis.set_major_formatter(date_formatter) 
ax.set_xlim(datemin, datemax)
max_freq = per_minute.max()
min_freq = per_minute.min()
ax.set_ylim(min_freq-100, max_freq+100) 
ax.plot(per_minute.index, per_minute)

display(fig)